In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier as XGB
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMClassifier as LGBM
from lightgbm import LGBMRegressor as LGBMR
from pprint import pprint

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Helper Functions

In [ ]:
def get_score(y, predicted):
    N = 5
    W = np.zeros((N,N))
    O = np.zeros((N,N))
    E = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            W[i,j] = (np.square(i-j)) / (np.square(N-1))
    for i in range(len(y)):
        O[y[i],predicted[i]] += 1
        E[y[i],y[i]] +=1
    total = np.sum(E)
    EE = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            EE[i,j] = E[i,i]/total*E[j,j]
    result = 1-np.sum(W*O)/np.sum(W*EE)
    return result

# Preparation

In [ ]:
train = pd.read_csv('../input/train/train.csv')
train = train.drop(columns = ['Name', 'RescuerID', 'PetID', 'Description'])
train = train[np.logical_not(np.logical_and(train['Breed1'] == 0, train['Breed2'] != 0))]
train_x = train.drop(columns = ['AdoptionSpeed'])
train_y = train['AdoptionSpeed']

In [ ]:
test = pd.read_csv('../input/test/test.csv')
test_x = test.drop(columns = ['Name', 'RescuerID', 'PetID', 'Description'])
pet_id = test['PetID']

In [ ]:
scorer = make_scorer(get_score, greater_is_better = True)

# Fitting

In [ ]:
LGBM_PARAMS = {
    'num_leaves': 100,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample_for_bin': 2000,
    'min_child_samples': 25,
    'reg_lambda': 0.03
}
lgbm = LGBM()
lgbm.fit(train_x, train_y)

# Prediction

In [ ]:
predict = lgbm.predict(test_x)
result = pd.DataFrame({'PetID': pet_id, 'AdoptionSpeed': predict})

In [ ]:
result.to_csv('submission.csv', index = False)